# 导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('/hy-tmp/self-llm/dataset/hate_test/train_json/train_json.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾',
  '请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾',
  '请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾'],
 'input': ['没爹的黑孩到处扔',
  '人伦人伦，没听说过狗伦，所以人作为高等的生物受到的束缚就多，狗没那么高等，受到的束缚就低，人大可不必学狗',
  '就最近来说有两个事，lol某主播的前女友发文称其飞黄腾达后抛弃她。我个人的想法是看男方的回应，本来恋爱期间有些东西是说不清的。结果小作文来了，一群人不等男方回应直接定论。如果男方承

# 处理数据集

In [4]:
tokenizer = AutoTokenizer.from_pretrained('/hy-tmp/model/qwen/Qwen2.5-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='/hy-tmp/model/qwen/Qwen2.5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_

In [5]:
def process_func(example):
    MAX_LENGTH = 4096    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要进行一个细粒度中文仇恨识别任务<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 4000/4000 [00:10<00:00, 398.94 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4000
})

In [7]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在你要进行一个细粒度中文仇恨识别任务<|im_end|>\n<|im_start|>user\n请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾没爹的黑孩到处扔<|im_end|>\n<|im_start|>assistant\n没爹的黑孩 | 到处扔 | Racism | hate [END]<|endoftext|>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'人 | 大可不必学狗 | non-hate | non-hate [END]<|endoftext|>'

# 创建模型

In [9]:
import torch

model = AutoModelForCausalLM.from_pretrained('/hy-tmp/model/qwen/Qwen2.5-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [10]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [11]:
model.dtype

torch.bfloat16

In [12]:
!nvidia-smi

Tue Feb 25 17:59:04 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.03             Driver Version: 535.216.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A800 80GB PCIe          On  | 00000000:37:00.0 Off |                    0 |
| N/A   40C    P0              69W / 300W |  15661MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# lora 

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'v_proj', 'q_proj', 'up_proj', 'gate_proj', 'k_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/hy-tmp/model/qwen/Qwen2.5-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'v_proj', 'q_proj', 'up_proj', 'gate_proj', 'k_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


# 配置训练参数

In [30]:
args = TrainingArguments(
    output_dir="./output/Qwen2.5_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [31]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [32]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.207900
20,0.329300


KeyboardInterrupt: 

In [16]:
!nvidia-smi

Tue Feb 25 17:59:19 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.03             Driver Version: 535.216.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A800 80GB PCIe          On  | 00000000:37:00.0 Off |                    0 |
| N/A   40C    P0              69W / 300W |  15699MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# 合并加载模型

In [21]:
import json

# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = '/hy-tmp/self-llm/dataset/hate_test/train_json/test_json.json'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取并解析 JSON 数据
        test_data = json.load(file)
        # 打印读取的数据
        print(test_data[0])
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except json.JSONDecodeError:
    print(f"文件 {file_path} 不是有效的 JSON 格式。")

{'instruction': '请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾', 'id': 4169, 'input': '飞周兄弟还不认亲？兄弟啊，还袭击？', 'output': ''}


In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/hy-tmp/model/qwen/Qwen2.5-7B-Instruct/'
lora_path = './output/Qwen2.5_instruct_lora/checkpoint-750' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)



Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


{'instruction': '请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾',
 'id': 4169,
 'input': '飞周兄弟还不认亲？兄弟啊，还袭击？',
 'output': ''}

In [ ]:
from tqdm import tqdm

# 假设 test_data、tokenizer 和 model 已经定义
for d in tqdm(test_data, desc="Processing test data"):
    prompt = d['input']
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": d['instruction']},{"role": "user", "content": prompt}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True
                                           ).to('cuda')
    
    
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        d['output'] = tokenizer.decode(outputs[0], skip_special_tokens=True)

Processing test data:   1%|▏         | 25/2000 [00:28<43:50,  1.33s/it]

In [ ]:
# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = '/hy-tmp/self-llm/dataset/hate_test/res/test_json.json'
try:
    with open(file_path, 'w', encoding='utf-8') as file:
        # 将 list 转换为 JSON 格式并写入文件
        json.dump(test_data, file, ensure_ascii=False, indent=4)
    print(f"数据已成功保存到 {file_path}")
except Exception as e:
    print(f"保存文件时出现错误: {e}")